In [1]:
# from kaggle.api.kaggle_api_extended import KaggleApi
# import kagglehub
# import kaggle
# Initiliaze API

In [1]:

from sklearn import *
import pandas as pd
import seaborn as sns
# Donwload Dataset via API unzipped
# api.dataset_download_files(dataset="prathamtripathi/drug-classification", unzip=True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
lol_df = pd.read_csv("Challenger_Ranked_Games_15minute.csv")
lol_df.shape

(26834, 51)

In [3]:
# Count the number of differences
difference_count = (lol_df['blueWins'] != lol_df['redWins']).sum()

print(f"Number of differences: {difference_count}")


Number of differences: 26834


In [5]:
#lol_df = lol_df.drop(columns=['redFirstBlood', 'blueFirstBlood'])
lol_df = lol_df.drop_duplicates()

In [6]:
lol_df = pd.get_dummies(lol_df)
lol_df.shape

(26779, 156)

# Dropping features which are highly correlated with each other

In [7]:
# Defining correlation Matrix
lol_corr = lol_df.corr()
lol_corr.head()

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,...,"redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON', 'AIR_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",redDragnoType_['WATER_DRAGON'],redDragnoType_[]
gameId,1.000000,-0.005397,0.001830,-0.009425,0.040736,0.040736,-0.076390,-0.041898,NaN,0.001016,...,-0.001389,0.008220,-0.002263,0.002762,-0.004503,-0.006312,-0.006266,-0.024747,-0.003234,0.016960
blueWins,-0.005397,1.000000,0.225555,0.261220,0.162420,0.162420,0.160936,0.137863,NaN,0.279579,...,-0.008202,-0.072166,-0.006112,-0.069009,-0.006112,-0.006114,-0.081301,0.010583,-0.047865,0.296870
blueTotalGolds,0.001830,0.225555,1.000000,0.361546,0.919173,0.919173,-0.203182,-0.378317,NaN,0.917561,...,0.051194,-0.043565,0.025898,-0.039175,0.027971,0.046982,-0.040577,0.017233,-0.074038,0.274931
blueCurrentGolds,-0.009425,0.261220,0.361546,1.000000,0.303369,0.303369,-0.044676,-0.099753,NaN,0.377635,...,0.006676,-0.039992,0.013962,-0.036403,0.000323,0.001936,-0.024224,0.002096,-0.045992,0.190912
blueTotalLevel,0.040736,0.162420,0.919173,0.303369,1.000000,1.000000,-0.178894,-0.388435,NaN,0.800904,...,0.033329,-0.036256,0.015903,-0.031122,0.017331,0.021814,-0.034913,0.010237,-0.071560,0.251489


Iterate through the correlation matrix

In [8]:
# Threshold to consider highly correlated features
high_threshold = 0.9

# Find list pairings of highly correlated features (excluding 'blueWins')
high_corr_pairs = []
for i in range(len(lol_corr.columns)):
    for j in range(i + 1, len(lol_corr.columns)):
        if abs(lol_corr.iloc[i, j]) > high_threshold and lol_corr.index[i] != 'blueWins' and lol_corr.columns[j] != 'blueWins':
            high_corr_pairs.append([lol_corr.index[i], lol_corr.columns[j]])

# Compare correlations with 'blueWins' and determine which features to drop
features_to_drop = []
for pair in high_corr_pairs:
    feature1, feature2 = pair
    if abs(lol_corr.loc[feature1, 'blueWins']) >= abs(lol_corr.loc[feature2, 'blueWins']):
        features_to_drop.append(feature2)  # Drop the feature with lower correlation with blueWins
    else:
        features_to_drop.append(feature1)

# Remove duplicates from the features_to_drop list
features_to_drop = list(set(features_to_drop))

In [9]:
# Dropping correlated features
lol_df_dropped = lol_df.drop(columns=features_to_drop)
lol_df_dropped = lol_df_dropped.drop(columns="gameId")

In [10]:
lol_df_dropped.shape

(26779, 143)

# Dropping features which are lowly correlated with target

Iterate through the correlation matrix

In [11]:
# Dropping features with low correlation with target

# Defining new correlation Matrix
lol_corr = lol_df_dropped.corr()


#
# Threshold to consider low correlation features with 'blueWins'
low_threshold = 0.05

# Find features with correlation below the threshold with 'blueWins'
low_corr_features = []
for feature in lol_corr.index:
    if feature != 'blueWins' and abs(lol_corr.loc[feature, 'blueWins']) < low_threshold:
        low_corr_features.append(feature)

# Output the list of features with low correlation to 'blueWins'

In [12]:
lol_df_dropped_2 = lol_df_dropped.drop(columns=low_corr_features)

In [13]:
lol_df_dropped_2.shape

(26779, 68)

In [30]:
lol_df_dropped_2.redWins

0        1
1        0
2        1
3        1
4        0
        ..
26829    1
26830    1
26831    1
26832    0
26833    0
Name: redWins, Length: 26779, dtype: int64

# KNN Machin Learning

In [32]:
features = lol_df_dropped_2.drop(columns=['blueWins','redWins'])
target = lol_df_dropped_2["blueWins"]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

#Normalize Data after Train Split
normalizer = MinMaxScaler() #define normalizer

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train) # Normalize 80% training dats
X_test_norm = normalizer.transform(X_test) # Normalize 20% Testing Data

#Apply to test and training data
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)


In [34]:
knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X_train_norm, y_train)

KNeighborsClassifier(n_neighbors=10)

In [35]:
knn.score(X_test_norm, y_test)

0.7378640776699029

In [36]:
knn.predict(X_test_norm)

array([0, 0, 1, ..., 1, 0, 1])

<h1>DECISION TREE</h1>

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize and fit the model
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train_norm, y_train)

# Make predictions
y_pred_tree = tree_clf.predict(X_test_norm)
y_pred_tree

array([0, 1, 1, ..., 0, 0, 1])

In [20]:
tree_clf.score(X_test_norm, y_test)

1.0

In [21]:
accuracy = accuracy_score(y_test, y_pred_tree)
accuracy

1.0

<h1>Random Forest</h1>

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and fit the model
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_norm, y_train)

# Make predictions
y_pred_rf = rf_clf.predict(X_test_norm)
y_pred_rf

accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_rf

1.0

<h1>Support Vector Machine</h1>

In [23]:
from sklearn.svm import SVC

# Initialize and fit the model
svm_clf = SVC(probability=True)  # Set probability=True to enable probability predictions
svm_clf.fit(X_train_norm, y_train)

# Make predictions
y_pred_svm = svm_clf.predict(X_test_norm)
y_pred_svm

array([0, 1, 1, ..., 0, 0, 1])

In [24]:
# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_svm

1.0